In [1]:
import sirf.STIR as STIR
import brainweb
from tqdm.auto import tqdm
import numpy

In [2]:
fname, url= sorted(brainweb.utils.LINKS.items())[0]
files = brainweb.get_file(fname, url, ".")
data = brainweb.load_file(fname)

brainweb.seed(1337)

In [3]:
for f in tqdm([fname], desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(f, petNoise=1, t1Noise=0.75, t2Noise=0.75, petSigma=1, t1Sigma=1, t2Sigma=1)
fdg_arr = vol['PET']

mMR ground truths: 100%|██████████| 1/1 [00:00<00:00,  4.10subject/s]


In [4]:
# Select central slice
central_slice = fdg_arr.shape[0]//2
fdg_arr = fdg_arr[central_slice, :, :]

# Select a central ROI with 120x120
idim = [120,120]
offset = (numpy.array(fdg_arr.shape) - numpy.array(idim)) // 2
fdg_arr = fdg_arr[offset[0]:offset[0]+idim[0], offset[1]:offset[1]+idim[1]]

# Now we make sure our image is of shape (1, 120, 120) 
fdg_arr = fdg_arr[numpy.newaxis,...]

In [5]:
templ_sino = STIR.AcquisitionData("template_sinogram.hs") # create an empty sinogram using a template

In [6]:
im = STIR.ImageData(templ_sino)
dim = fdg_arr.shape
voxel_size=im.voxel_sizes()
im.initialise(dim,(voxel_size[0]*2, voxel_size[1], voxel_size[2]))
fdg = im.clone().fill(fdg_arr)


INFO: Determined voxel size by dividing default_bin_size (2.03125) by zoom


In [7]:
acq_model_matrix = STIR.SPECTUBMatrix()
acq_model_matrix.set_resolution_model(0,0,full_3D=False)
am = STIR.AcquisitionModelUsingMatrix(acq_model_matrix)
am.set_up(templ_sino, fdg)


INFO: No correction for PSF. Parallel geometry

Parameters of SPECT UB matrix: (in cm)
Image grid side row: 120	col: 120	transverse voxel_size: 0.203125
Number of slices: 1	slice_thickness: 0.203125
Number of bins: 125	bin size: 0.203125	axial size: 0.203125
Number of angles: 75	Angle increment: -4.8	First angle: 0
Number of subsets: 75
Rotation radii: {25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6, 25.6}
Minimum weight: 0


INFO: Done estimating size of matrix. Execution (CPU) time 0.1 s 


In [8]:
sino = am.forward(fdg)


INFO: Processing view 0 of segment 0



Going ahead anyway.



INFO: Computing matrix elements for view 0

INFO: total number of non-zero weights in this view: 28925, estimated size: 0.287771 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0.01 (s)

INFO: Processing view 1 of segment 0

INFO: Computing matrix elements for view 1

INFO: total number of non-zero weights in this view: 29937, estimated size: 0.297422 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 2 of segment 0

INFO: Computing matrix elements for view 2

INFO: total number of non-zero weights in this view: 30529, estimated size: 0.303068 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 3 of segment 0

INFO: Computing matrix elements for view 3

INFO: total number of non-zero weights in this view: 31030, e

In [9]:
bp = am.backward(sino)

backprojecting...
INFO: Processing view 0 of segment 0

INFO: Computing matrix elements for view 0

INFO: total number of non-zero weights in this view: 28925, estimated size: 0.287771 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 1 of segment 0

INFO: Computing matrix elements for view 1

INFO: total number of non-zero weights in this view: 29937, estimated size: 0.297422 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 2 of segment 0

INFO: Computing matrix elements for view 2

INFO: total number of non-zero weights in this view: 30529, estimated size: 0.303068 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 3 of segment 0

INFO: Computing matrix elements for view 3

INFO: tot

In [10]:
sino = am.forward(bp)


Going ahead anyway.



INFO: Processing view 0 of segment 0

INFO: Computing matrix elements for view 0

INFO: total number of non-zero weights in this view: 28925, estimated size: 0.287771 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 1 of segment 0

INFO: Computing matrix elements for view 1

INFO: total number of non-zero weights in this view: 29937, estimated size: 0.297422 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 2 of segment 0

INFO: Computing matrix elements for view 2

INFO: total number of non-zero weights in this view: 30529, estimated size: 0.303068 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 3 of segment 0

INFO: Computing matrix elements for view 3

INFO: total number of non-

In [11]:
bp = am.backward(sino)

backprojecting...
INFO: Processing view 0 of segment 0

INFO: Computing matrix elements for view 0

INFO: total number of non-zero weights in this view: 28925, estimated size: 0.287771 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 1 of segment 0

INFO: Computing matrix elements for view 1

INFO: total number of non-zero weights in this view: 29937, estimated size: 0.297422 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 2 of segment 0

INFO: Computing matrix elements for view 2

INFO: total number of non-zero weights in this view: 30529, estimated size: 0.303068 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 3 of segment 0

INFO: Computing matrix elements for view 3

INFO: tot

In [12]:
sino = am.forward(bp)

segment 0

INFO: Computing matrix elements for view 70

INFO: total number of non-zero weights in this view: 31790, estimated size: 0.315094 MB

INFO: Weight matrix calculation done. time 0.01 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0.01 (s)

INFO: Processing view 71 of segment 0

INFO: Computing matrix elements for view 71

INFO: total number of non-zero weights in this view: 31449, estimated size: 0.311842 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 72 of segment 0

INFO: Computing matrix elements for view 72

INFO: total number of non-zero weights in this view: 31030, estimated size: 0.307846 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 73 of segment 0

INFO: Computing matrix elements for view 73

INFO: total number of non-zero weights in


Going ahead anyway.



INFO: Processing view 0 of segment 0

INFO: Computing matrix elements for view 0

INFO: total number of non-zero weights in this view: 28925, estimated size: 0.287771 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 1 of segment 0

INFO: Computing matrix elements for view 1

INFO: total number of non-zero weights in this view: 29937, estimated size: 0.297422 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 2 of segment 0

INFO: Computing matrix elements for view 2

INFO: total number of non-zero weights in this view: 30529, estimated size: 0.303068 MB

INFO: Weight matrix calculation done. time 0 (s)

INFO: Total time after transfering to ProjMatrixElemsForOneBin. time 0 (s)

INFO: Processing view 3 of segment 0

INFO: Computing matrix elements for view 3

INFO: total number of non-

In [ ]:
sino = am.forward(fdg)
bp = am.backward(sino)
sino = am.forward(bp)
bp = am.backward(sino)